In [5]:
import pandas as pd
import csv
import re
from sklearn.preprocessing import MinMaxScaler

In [6]:
file = 'data.csv' #large file with trade data
isocodefile='isocode.csv'
distancefile='dist_capital.csv'

In [8]:
trade_data = pd.read_csv(file)
isocode=pd.read_csv(isocodefile)
distance=pd.read_csv(distancefile)
trade_data['CountryPair'] = trade_data['Reporter'] + trade_data['Partner'] #use this column to join datasets later if needed
trade_data['Trade_Indicator'] = trade_data['Ind1'] + trade_data['Ind4'] #share of exports from reporter country + share of imports from partner country
trade_data = trade_data[trade_data['L3']=='000'].copy()
trade_data = trade_data[['Partner', 'Reporter', 'Trade_Indicator', 'CountryPair', 'L3']]

In [9]:
pcc = trade_data['Partner'].str.contains("C")
trade_data = trade_data[pcc]
rcc = trade_data['Reporter'].str.contains("C")
trade_data = trade_data[rcc]

In [10]:
trade_data["Partner"] = trade_data["Partner"].str.replace(pat=r'^C', repl=r'', regex=True)
trade_data["Reporter"] = trade_data["Reporter"].str.replace(pat=r'^C', repl=r'', regex=True)

In [11]:
trade_data['Partner'] = pd.to_numeric(trade_data['Partner'])
trade_data['Reporter'] = pd.to_numeric(trade_data['Reporter'])
trade=pd.merge(trade_data,isocode, left_on='Partner',right_on='Numeric code',how='inner')
trade.rename(columns = {'Alpha-3 code' : 'partneriso'}, inplace = True)
trade=trade[['partneriso','Reporter','Trade_Indicator','CountryPair','L3']]

In [12]:
trade=pd.merge(trade,isocode, left_on='Reporter',right_on='Numeric code',how='inner')
trade.rename(columns = {'Alpha-3 code' : 'reporteriso'}, inplace = True)
trade=trade[['partneriso','reporteriso','Trade_Indicator','CountryPair','L3']]

In [13]:
#normalize
trade['tradenorm']=trade['Trade_Indicator']/abs(trade['Trade_Indicator'].max())
distance['disnorm']=distance['distcap']/abs(distance['distcap'].max())

In [14]:
normdata = pd.merge(left=trade, right=distance, how='inner', left_on=['partneriso','reporteriso'], right_on=['iso_d','iso_o'])
normdata['score']=normdata['tradenorm']-normdata['disnorm']
normdata=normdata[['partneriso','reporteriso','Trade_Indicator','CountryPair','L3','tradenorm','disnorm','score']]
normdata = normdata.sort_values(by='score' ,ascending=False)

In [16]:
normdata.to_csv('normdata.csv')